In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *
from Utils.utils import *

from pyspark.sql.functions import (
    col,                    
    when,                   
    lit,                    
    trim,                   
    upper,                 
    lower,                         
    coalesce,              
    concat,                            
    substring,             
    length,                
    cast,                  
    round,                
    sum,                   
    count,                 
    avg,                   
    max,                   
    min,                   
    countDistinct,         
    current_timestamp,     
    year,                  
    month,                 
    datediff,              
    to_date,               
    date_format,           
    rank,                  
    dense_rank,            
    lag,                  
    lead,
    decode,
    encode ,
    regexp_replace,
    lpad, 
    translate,
    collect_list,
    explode,
    split,
    sha2,
    concat_ws             
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType,
    ByteType
)

from pyspark.sql import Window, DataFrame

apply_storage_config(spark)

In [0]:
dbutils.fs.ls(bronze_path)

Definição dos paths

In [0]:
path_storage_bronze = f"{bronze_path}"
path_storage_silver = f"{silver_path}"


## Limpeza e tratamentos

### Limpeza e Validacao de Dados

- Remover colunas não utilizavéis
- Padronizar formatos

### Transformacao de Tipos de Dados

- Dados brutos vem como STRING
- Para fazer calculos, precisamos de INT, DOUBLE, etc
- Economiza espaco de armazenamento
- Melhora performance de queries

#### Tratamentos NCM

In [0]:
# NCM - Tratamento e filtros

# Abrindo o arquivo do NCM na Bronze
df_ncm = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/ncm/")

# Recuperando as colunas
columns = df_ncm.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm = latin1_to_utf8(df_ncm, ["NO_NCM_POR"])

# 2. strip/trim
df_ncm = strip_df(df_ncm, ["NO_NCM_POR"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm = normalize_blanks_to_null(df_ncm, ["NO_NCM_POR"])

# 4. LPAD dos NCM
df_ncm = df_ncm.withColumn("CO_NCM", lpad(col('CO_NCM'), 8, "0") )
df_ncm = df_ncm.withColumn("CO_SH6", lpad(col('CO_SH6'), 6, "0") )

display(df_ncm)

In [0]:
display(df_ncm)

In [0]:
df_ncm.printSchema()

In [0]:
display(df_ncm.groupby("CO_NCM", "CO_SH6").count())

In [0]:
# Removendo as colunas NO_NCM_ESP, NO_NCM_ING
df_ncm_filtrado = df_ncm.drop("NO_NCM_ESP", "NO_NCM_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_ncm_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_tratado")
)

In [0]:
df_ncm_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_tratado")

#### Tratamentos NCM SH

In [0]:
df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")

In [0]:
display(df_ncm_sh)

In [0]:
# ncm_sh - Tratamento e filtros

# Abrindo o arquivo do NCM_SH na Bronze
df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")

# Recuperando as colunas
columns = df_ncm_sh.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_sh = latin1_to_utf8(df_ncm_sh, ["NO_ncm_sh_POR"])

# 2. strip/trim
df_ncm_sh = strip_df(df_ncm_sh, ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_sh = normalize_blanks_to_null(df_ncm_sh, ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"])

# 4. LPAD dos ncm_sh
df_ncm_sh = df_ncm_sh.withColumn("CO_SH6", lpad(col('CO_SH6'), 6, "0") )
df_ncm_sh = df_ncm_sh.withColumn("CO_SH4", lpad(col('CO_SH4'), 4, "0") )
df_ncm_sh = df_ncm_sh.withColumn("CO_SH2", lpad(col('CO_SH2'), 2, "0") )


In [0]:
df_ncm_sh_filtrado = df_ncm_sh.drop('NO_SH6_ESP','NO_SH6_ING','NO_SH4_ESP','NO_SH4_ING','NO_SH2_ESP','NO_SH2_ING','NO_SEC_ESP','NO_SEC_ING','ingestion_dt','tipo_operacao','origin_path_name')

display(df_ncm_sh_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_sh_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_sh_tratado")
)

In [0]:
df_ncm_sh_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_sh_tratado")

#### Tratamentos NCM ISIC

In [0]:
df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic/")

In [0]:
display(df_ncm_isic)

In [0]:
df_ncm_isic.columns

In [0]:
# ncm_isic - Tratamento e filtros

# Abrindo o arquivo do ncm_isic na Bronze
df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic/")

# Recuperando as colunas
columns = df_ncm_isic.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_isic = latin1_to_utf8(df_ncm_isic, ["NO_ncm_isic_POR"])

# 2. strip/trim
df_ncm_isic = strip_df(df_ncm_isic, ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_isic = normalize_blanks_to_null(df_ncm_isic, ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"])

# 4. LPAD dos ncm_isic
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_CLASSE", lpad(col('CO_ISIC_CLASSE'), 4, "0") )
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_GRUPO", lpad(col('CO_ISIC_GRUPO'), 3, "0") )
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_DIVISAO", lpad(col('CO_ISIC_DIVISAO'), 2, "0") )


display(df_ncm_isic)

In [0]:

df_ncm_isic_filtrado = df_ncm_isic.drop('NO_ISIC_CLASSE_ESP','NO_ISIC_CLASSE_ING','NO_ISIC_GRUPO_ESP','NO_ISIC_GRUPO_ING','NO_ISIC_DIVISAO_ESP','NO_ISIC_DIVISAO_ING','NO_ISIC_SECAO_ESP','NO_ISIC_SECAO_ING','ingestion_dt','tipo_operacao','origin_path_name')

display(df_ncm_isic_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_ncm_isic_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_isic_tratado")
)

In [0]:
df_ncm_isic_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_isic_tratado")

##### Tratamentos NCM_CGCE

In [0]:
df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce/")

In [0]:
display(df_ncm_cgce)

In [0]:
df_ncm_cgce.columns

In [0]:
# ncm_cgce - Tratamento e filtros

# Abrindo o arquivo do ncm_cgce na Bronze
df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce/")

# Recuperando as colunas
columns = df_ncm_cgce.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_cgce = latin1_to_utf8(df_ncm_cgce, ["NO_ncm_cgce_POR"])

# 2. strip/trim
df_ncm_cgce = strip_df(df_ncm_cgce, ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_cgce = normalize_blanks_to_null(df_ncm_cgce, ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"])

# 4. LPAD dos ncm_cgce
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N3", lpad(col('CO_CGCE_N3'), 3, "0") )
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N2", lpad(col("CO_CGCE_N2"), 2, "0") )
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N1", lpad(col('CO_CGCE_N1'), 1, "0") )


display(df_ncm_cgce)

In [0]:

# df_ncm_cgce_filtrado
df_ncm_cgce_filtrado = df_ncm_cgce.drop('NO_CGCE_N3_ESP','NO_CGCE_N3_ING','NO_CGCE_N2_ESP','NO_CGCE_N2_ING','NO_CGCE_N1_ESP','NO_CGCE_N1_ING','ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_cgce_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_cgce_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_cgce_tratado")
)

In [0]:
df_ncm_cgce_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_cgce_tratado")

#### Tratamentos NCM_FAT_AGREG

In [0]:
# ncm_fat_agreg - Tratamento e filtros

# Abrindo o arquivo do ncm_fat_agreg na Bronze
df_ncm_fat_agreg = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_fat_agreg/")

# Recuperando as colunas
columns = df_ncm_fat_agreg.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_fat_agreg = latin1_to_utf8(df_ncm_fat_agreg, ["NO_ncm_fat_agreg_POR"])

# 2. strip/trim
df_ncm_fat_agreg = strip_df(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_fat_agreg = normalize_blanks_to_null(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])

# 4. LPAD dos ncm_fat_agreg
df_ncm_fat_agreg = df_ncm_fat_agreg.withColumn("CO_FAT_AGREG", lpad(col('CO_FAT_AGREG'), 2, "0") )

# 5. Limpeza de acentos
df_ncm_fat_agreg = remove_accents(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])


display(df_ncm_fat_agreg)

In [0]:

# df_ncm_fat_agreg_filtrado
df_ncm_fat_agreg_filtrado = df_ncm_fat_agreg.drop('NO_FAT_AGREG_GP','ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_fat_agreg_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_fat_agreg_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_fat_agreg_tratado")
)

In [0]:
df_ncm_fat_agreg_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_fat_agreg_tratado")

#### Tratamentos PAIS

In [0]:
df_pais = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais")


In [0]:
display(df_pais)

In [0]:
# pais - Tratamento e filtros

# Abrindo o arquivo do pais na Bronze
df_pais = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais")

# Recuperando as colunas
columns = df_pais.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_pais = latin1_to_utf8(df_pais, ["NO_PAIS"])

# 2. strip/trim
df_pais = strip_df(df_pais, ["NO_PAIS"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_pais = normalize_blanks_to_null(df_pais, ["NO_PAIS"])

# 4. LPAD dos pais
df_pais =df_pais.withColumn("CO_PAIS", col("CO_PAIS").cast("int"))
df_pais =df_pais.withColumn("CO_PAIS_ISON3", col("CO_PAIS_ISON3").cast("int"))

display(df_pais)

In [0]:
# Removendo as colunas _ESP, _ING
df_pais_filtrado = df_pais.drop("NO_PAIS_ESP", "NO_PAIS_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_pais_filtrado.limit(10))

In [0]:
display(df_pais_filtrado.filter(col("CO_PAIS").isNull()))


In [0]:
# Salvando no storage da silver
(
    df_pais_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/pais_tratado")
)

In [0]:
df_pais_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.pais_tratado")

#### Tratamentos PAIS_BLOCO

In [0]:
# pais_bloco - Tratamento e filtros

# Abrindo o arquivo do pais_bloco na Bronze
df_pais_bloco = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais_bloco")

# Recuperando as colunas
columns = df_pais_bloco.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_pais_bloco = latin1_to_utf8(df_pais_bloco, ["NO_pais_bloco"])

# 2. strip/trim
df_pais_bloco = strip_df(df_pais_bloco, ["NO_BLOCO"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_pais_bloco = normalize_blanks_to_null(df_pais_bloco, ["NO_BLOCO"])

# 4. CAST INT dos pais_bloco
df_pais_bloco = df_pais_bloco.withColumn("CO_PAIS", col("CO_PAIS").cast("int"))
df_pais_bloco = df_pais_bloco.withColumn("CO_BLOCO", col("CO_BLOCO").cast("int"))

display(df_pais_bloco)

In [0]:
# Removendo as colunas _ESP, _ING
df_pais_bloco_filtrado = df_pais_bloco.drop("NO_BLOCO_ESP", "NO_BLOCO_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_pais_bloco_filtrado.limit(10))

In [0]:
# Salvando no storage da silver
(
    df_pais_bloco_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/pais_bloco_tratado")
)

In [0]:
df_pais_bloco_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.pais_bloco_tratado")

#### Tratamentos UF

In [0]:
df_uf = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf")

In [0]:
display(df_uf)

In [0]:
# uf - Tratamento e filtros

# Abrindo o arquivo do uf na Bronze
df_uf = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf")

# Recuperando as colunas
columns = df_uf.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_uf = latin1_to_utf8(df_uf, ["NO_uf"])

# 2. strip/trim
df_uf = strip_df(df_uf, ["SG_UF", "NO_UF", "NO_REGIAO"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_uf = normalize_blanks_to_null(df_uf, ["NO_BLOCO"])

# 4. LPAD dos uf
#df_uf = df_uf.withColumn("CO_UF", lpad(col('CO_UF'), 2, "0") )
df_uf = df_uf.withColumn("CO_UF", col("CO_UF").cast(ByteType()))

display(df_uf)

In [0]:
df_uf_filtrado = df_uf.drop('ingestion_dt','tipo_operacao','origin_path_name')
display(df_uf_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_uf_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/uf_tratado")
)

In [0]:
df_uf_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.uf_tratado")

#### Tratamentos UF_MUN

In [0]:
display(df_uf_mun)

In [0]:
# uf_mun - Tratamento e filtros

# Abrindo o arquivo do uf_mun na Bronze
df_uf_mun = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf_mun")

# Recuperando as colunas
columns = df_uf_mun.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_uf_mun = latin1_to_utf8(df_uf_mun, ["NO_uf_mun"])

# 2. strip/trim
df_uf_mun = strip_df(df_uf_mun, ["NO_MUN", "SG_UF"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_uf_mun = normalize_blanks_to_null(df_uf_mun, ["NO_MUN", "SG_UF"])

# 4. LPAD dos uf_mun
df_uf_mun = df_uf_mun.withColumn("CO_MUN_GEO", col("CO_MUN_GEO").cast("int"))

display(df_uf_mun)

In [0]:
df_uf_mun_filtrado = df_uf_mun.drop('ingestion_dt','tipo_operacao','origin_path_name', 'NO_MUN_MIN')
display(df_uf_mun_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_uf_mun_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/uf_mun_tratado")
)

In [0]:
df_uf_mun_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.uf_mun_tratado")